In [ ]:
import json
from websockets import connect
import websocket
import pandas as pd 

In [ ]:
# !pip install websocket-client

In [ ]:
# !pip install websocket

In [ ]:
# # Replace with your actual API key
# # API_KEY = "YOUR_API_KEY"

# async def get_candlestick_data():
#     uri = "wss://coinswitch.co/api/ws"  # WebSocket endpoint URL
#     async with connect(uri) as websocket:
#         # Subscribe to the channel for the desired currency pair
#         await websocket.send(json.dumps({"setOp": "subscribe", "args": ["spot-btcusdt"]}))

#         while True:
#             try:
#                 data = await websocket.recv()
#                 # Parse JSON data
#                 parsed_data = json.loads(data)

#                 if "k" in parsed_data:  # Check for candlestick data
#                     # Extract relevant information (replace with specific keys you need)
#                     open_price = parsed_data["k"]["o"]
#                     high_price = parsed_data["k"]["h"]
#                     low_price = parsed_data["k"]["l"]
#                     close_price = parsed_data["k"]["c"]
#                     volume = parsed_da["k"]["v"]

#                     # Process the data (e.g., print, store in database)
#                     print(f"Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}, Volume: {volume}")

#             except Exception as e:
#                 print(f"Error: {e}")
#                 break  # Handle errors and disconnect if needed

# if __name__ == "__main__":
#     import asyncio
#     asyncio.get_event_loop().run_until_complete(get_candlestick_data())


# Fetching the real time data from Binance websocket server

In [ ]:
######------- 100% working----------------
import json
import websocket
import pandas as pd
from datetime import datetime
import time

# Global DataFrame to store real-time data
df = pd.DataFrame(columns=['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Function to fetch real-time data and store in DataFrame
def fetch_real_time_data(assets):
    def on_message(ws, message):
        global df
        message = json.loads(message)
        
        if 'data' in message and 'k' in message['data']:
            kline = message['data']['k']
            data = {
                'Date': datetime.fromtimestamp(kline['t'] / 1000),
                'Symbol': kline['s'],
                'Open': float(kline['o']),
                'High': float(kline['h']),
                'Low': float(kline['l']),
                'Close': float(kline['c']),
                'Volume': float(kline['v'])
            }
            df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
            print(data)
            # Sleep for 1 minute to store data at 1-minute intervals
            time.sleep(60)
    
    def on_error(ws, error):
        print(error)

    def on_close(ws, close_status_code, close_msg):
        print("### closed ###")

    def on_open(ws):
        print("### connected ###")

    assets = [coin.lower() + '@kline_1m' for coin in assets]
    assets = '/'.join(assets)

    socket = "wss://stream.binance.com:9443/stream?streams=" + assets
    ws = websocket.WebSocketApp(socket,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.on_open = on_open
    ws.run_forever()

# Example usage
fetch_real_time_data(['BTCUSDT'])


In [ ]:
df

# Real time crypto data fetch from cryptocompare python library

In [ ]:
####---------- 100% working:-------------
import time
import pandas as pd
import cryptocompare
from datetime import datetime

# Global DataFrame to store real-time data
df = pd.DataFrame(columns=['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Function to fetch real-time data and store in DataFrame
def fetch_real_time_data(Symbol, interval=120):
    global df

    while True:
        # Fetch the current price data
        price = cryptocompare.get_price(symbol, currency='USD', full=True)
        
        if price and 'RAW' in price and symbol in price['RAW']:
            data = price['RAW'][symbol]['USD']
            timestamp = datetime.now()
            row = {
                'Date': timestamp,
                'Symbol': symbol,
                'Open': data['OPEN24HOUR'],
                'High': data['HIGH24HOUR'],
                'Low': data['LOW24HOUR'],
                'Close': data['PRICE'],
                'Volume': data['VOLUME24HOUR']
            }
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
            print(row)

        # Sleep for the specified interval (default is 2 minutes)
        time.sleep(interval)

# Example usage
fetch_real_time_data('BTC')


In [2]:
import time
import pandas as pd
import cryptocompare
from datetime import datetime
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display

# Initialize an empty DataFrame with column names
columns = ['timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(columns=columns)

# Create a Plotly FigureWidget for real-time updates
fig = go.FigureWidget(make_subplots(rows=1, cols=1, shared_xaxes=True))
candlestick = go.Candlestick(x=[], open=[], high=[], low=[], close=[], name='Candlestick')
fig.add_trace(candlestick, row=1, col=1)

# Update the layout to look more professional
fig.update_layout(
    title='Real-Time BTC/USD Candlestick Chart',
    xaxis_title='Timestamp',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,
    template='plotly_dark',
    showlegend=False,
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgba(255, 255, 255, 0.5)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgba(255, 255, 255, 0.7)',
        ),
    ),
    yaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgba(255, 255, 255, 0.5)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgba(255, 255, 255, 0.7)',
        ),
    ),
    plot_bgcolor='rgba(10, 10, 10, 1)',
    paper_bgcolor='rgba(10, 10, 10, 1)',
)

# Display the figure widget
display(fig)

# Function to fetch real-time data
def fetch_real_time_data(symbol, interval=60):
    global df

    while True:
        try:
            # Fetch the current price data
            price = cryptocompare.get_price(symbol, currency='USD', full=True)
            
            if price and 'RAW' in price and symbol in price['RAW']:
                data = price['RAW'][symbol]['USD']
                timestamp = datetime.now()
                new_row = pd.DataFrame([[timestamp, symbol, data['OPEN24HOUR'], data['HIGH24HOUR'], 
                                         data['LOW24HOUR'], data['PRICE'], data['VOLUME24HOUR']]], 
                                       columns=columns)
                if df.empty:
                    df = new_row  # Initialize the DataFrame if empty
                else:
                    df = pd.concat([df, new_row], ignore_index=True)
                
                print(new_row.to_dict(orient='records')[0])  # Print the new row

                # Update the candlestick chart
                update_candlestick_chart(df)
            
            # Sleep for the specified interval (default is 2 minutes)
            time.sleep(interval)
        
        except Exception as e:
            print(f"Error fetching data: {e}")
            time.sleep(interval)

# Function to update the candlestick chart using Plotly
def update_candlestick_chart(data):
    try:
        if data.empty:
            print("DataFrame is empty, cannot plot.")
            return
        
        with fig.batch_update():
            fig.data[0].x = data['timestamp']
            fig.data[0].open = data['open']
            fig.data[0].high = data['high']
            fig.data[0].low = data['low']
            fig.data[0].close = data['close']
        
    except Exception as e:
        print(f"Error updating candlestick chart: {e}")

# Example usage
fetch_real_time_data('BTC')
plot_candlestick_chart(df)

FigureWidget({
    'data': [{'close': [],
              'high': [],
              'low': [],
              'name': 'Candlestick',
              'open': [],
              'type': 'candlestick',
              'uid': '8567a2bb-f813-44a5-96d7-1b064e45ef94',
              'x': [],
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'paper_bgcolor': 'rgba(10, 10, 10, 1)',
               'plot_bgcolor': 'rgba(10, 10, 10, 1)',
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Real-Time BTC/USD Candlestick Chart'},
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'linecolor': 'rgba(255, 255, 255, 0.5)',
                         'linewidth': 2,
                         'rangeslider': {'visible': False},
                         'showgrid': False,
                         'showline': True,
                         'showticklabels': True,
                       

{'timestamp': Timestamp('2024-06-20 12:36:52.476106'), 'symbol': 'BTC', 'open': 65317.779070931, 'high': 65610.5925833765, 'low': 64656.8411976013, 'close': 65520.5516342051, 'volume': 11698.47859851}


KeyboardInterrupt: 

## EMA strategies for Purchase/sell in any stocks

In [ ]:
data.shape

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Sample data creation for demonstration purposes
# Replace 'crypto_prices.csv' with your actual file path and ensure it has 'Date', 'Open', 'High', 'Low', and 'Close' columns
# The following lines are for creating a sample dataset if you don't have one
# Comment out or remove these lines if using an actual dataset
dates = pd.date_range(start='2022-01-01', end='2023-01-01', freq='D')
close_prices = np.random.normal(loc=100, scale=10, size=len(dates))
data = pd.DataFrame({'Close': close_prices}, index=dates)
data['Open'] = data['Close'] + np.random.normal(loc=0, scale=2, size=len(dates))
data['High'] = data[['Close', 'Open']].max(axis=1) + np.random.normal(loc=0, scale=2, size=len(dates))
data['Low'] = data[['Close', 'Open']].min(axis=1) - np.random.normal(loc=0, scale=2, size=len(dates))
data['Date'] = data.index
data.to_csv('crypto_prices.csv')

# Load historical price data
data = pd.read_csv('crypto_prices.csv', parse_dates=['Date'], index_col='Date')

# Calculate EMAs
data['EMA_9'] = data['Close'].ewm(span=9, adjust=False).mean()
data['EMA_21'] = data['Close'].ewm(span=21, adjust=False).mean()
data['EMA_55'] = data['Close'].ewm(span=55, adjust=False).mean()

# Generate buy/sell signals
data['Signal'] = 0
data['Signal'] = np.where((data['EMA_9'] > data['EMA_21']) & (data['EMA_21'] > data['EMA_55']), 1, data['Signal'])
data['Signal'] = np.where((data['EMA_9'] < data['EMA_21']) & (data['EMA_21'] < data['EMA_55']), -1, data['Signal'])

# Create buy and sell columns
data['Buy'] = np.where(data['Signal'] == 1, data['Close'], np.nan)
data['Sell'] = np.where(data['Signal'] == -1, data['Close'], np.nan)

# Create candlestick chart using Plotly Express
fig = go.Figure(data=[go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name='Candlestick'
)])

# Add EMAs
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_9'], mode='lines', name='9-period EMA', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_21'], mode='lines', name='21-period EMA', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_55'], mode='lines', name='55-period EMA', line=dict(color='green')))

# Add buy and sell signals
fig.add_trace(go.Scatter(x=data.index, y=data['Buy'], mode='markers', marker=dict(symbol='triangle-up', color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=data.index, y=data['Sell'], mode='markers', marker=dict(symbol='triangle-down', color='red', size=10), name='Sell Signal'))

# Update layout with range slider and selector
fig.update_layout(
    title='Crypto Buy/Sell Signals with Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=1, label='YTD', step='year', stepmode='todate'),
                dict(count=1, label='1y', step='year', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

# Show the plot
fig.show()


# Experiments

In [7]:
import streamlit as st
import pandas as pd
import numpy as np
import ccxt
import plotly.graph_objects as go
from datetime import datetime, timedelta
import websocket
import json


# Function to fetch data from Binance
def fetch_data(symbol, timeframe, limit=200):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    return data

fetch_data('BTC/USDT','1m')

,timestamp,open,high,low,close,volume
0,2024-06-21 03:26:00,64795.75,64795.75,64791.87,64791.87,2.39310
1,2024-06-21 03:27:00,64791.86,64795.04,64780.93,64795.04,6.20268
2,2024-06-21 03:28:00,64795.04,64800.00,64795.04,64800.00,2.89898
3,2024-06-21 03:29:00,64799.99,64800.00,64780.00,64780.01,9.85012
4,2024-06-21 03:30:00,64780.01,64783.89,64759.50,64759.51,4.18765
...,...,...,...,...,...,...
195,2024-06-21 06:41:00,64659.99,64659.99,64619.69,64619.70,6.20570
196,2024-06-21 06:42:00,64619.70,64619.70,64619.69,64619.70,2.27686
197,2024-06-21 06:43:00,64619.70,64632.00,64619.69,64632.00,7.12148
198,2024-06-21 06:44:00,64631.99,64632.00,64631.99,64632.00,3.50223
